In [ ]:
%load_ext autoreload
%autoreload
import pandas as pd
import dataWrangling
from memoryReduction import *
import gc
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc, accuracy_score
import multiprocessing
sns.set()
print(multiprocessing.cpu_count())

In [ ]:
#df = dataWrangling.create_dt(is_train=True, first_day= 1000)

In [ ]:
'''
%timeit 

import multiprocessing as mp
importlib.reload(dataWrangling)
#p = mp.Pool(2)
df = dataWrangling.create_fea(df)
df = reduce_mem_usage(df)
df.to_pickle("data.pkl")

In [ ]:
df = pd.read_pickle("./dummy.pkl")

In [ ]:
df.dropna(inplace=True)

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + \
            ["event_name_1", "event_name_2", "event_type_1", "event_type_2"] +\
            ['year','month','week','day','dayofweek']
useless_cols = ["id", "date", "sales","d", "wm_yr_wk"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]
X_train = reduce_mem_usage(X_train)
y_train = reduce_mem_usage(y_train)

del df
gc.collect()


In [ ]:
df.head()

In [ ]:
'''
train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
fake_valid_inds = np.random.choice(len(X_train), 500000)
fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
                             free_raw_data=False)



In [ ]:
#taken from https://www.kaggle.com/bitit1994/parameter-grid-search-lgbm-with-scikit-learn
'''
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        'verbosity': 1,
        'num_iterations' : 25,
}
'''
# Initiate classifier to use
mdl = lgb.LGBMClassifier()

In [ ]:
gridParams = {
    'learning_rate': [0.005, 0.01,0.05],
#    'n_estimators': [8,16,24],
    'num_leaves': [12,48,86], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['poisson','regression'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    "sub_feature" : [0.9,0.8,0.7],
    "sub_row" : [0.75,0.8,0.85],
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75,0.8],
    'lambda_l1' : [0.1,0.05,0.15,0.5,1,1.2],
    "lambda_l2" : [0.1,0.05,0.15,0.5,1,1.2],
    "metric" : ['rmse'],
    'num_iterations' : [100],
    'verbosity': [1],
    "force_row_wise" : [True],
    "bagging_freq" : [1],
    }

grid = GridSearchCV(mdl, gridParams, verbose=1, n_jobs=4,cv = 4) ##remember to add cv
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

file = open("params.txt","w") 
file.writelines(grid.best_params_) 
file.close()

In [ ]:
train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
fake_valid_inds = np.random.choice(len(X_train), 500000)
fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
                             free_raw_data=False)
del X_train, y_train
gc.collect()

In [ ]:
params = {
    'learning_rate': grid.best_params_['colsample_bytree'],
    'num_leaves': grid.best_params_['num_leaves'], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : grid.best_params_['boosting_type'], # for better accuracy -> try dart
    'objective' : grid.best_params_['objective'],
    'max_bin': grid.best_params_['max_bin'], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : grid.best_params_['random_state'],
    "sub_feature" : grid.best_params_['sub_feature'],
    "sub_row" : grid.best_params_['sub_row'],
    'colsample_bytree' : grid.best_params_['colsample_bytree'],
    'subsample' : grid.best_params_['subsample'],
    'lambda_l1' : grid.best_params_['lambda_l1'],
    "lambda_l2" : grid.best_params_['lambda_l2'],
    "metric" : grid.best_params_['metric'],
    'num_iterations' : ["2500"],
    'verbosity': grid.best_params_['verbosity'],
    "force_row_wise" : grid.best_params_['force_row_wise'],
    "bagging_freq" : grid.best_params_['bagging_freq'],
}

#Initiate classifier to use
m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=100) 


In [ ]:
m_lgb.save_model("model.lgb")

del train_data,fake_valid_data

In [ ]:
h = 28 
max_lags = 366
tr_last = 1913
fday = datetime(2016,4, 25) 
te = create_dt(False)
cols = [f"F{i}" for i in range(1,29)]

for tdelta in range(0, 28):
    day = fday + timedelta(days=tdelta)
    print(day)
    tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
    create_fea(tst)
    tst = tst.loc[tst.date == day , train_cols]
    te.loc[te.date == day, "sales"] = m_lgb.predict(tst) # magic multiplier by kyakovlev



te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
te_sub.fillna(0., inplace = True)
te_sub.sort_values("id", inplace = True)
te_sub.reset_index(drop=True, inplace = True)
te_sub.to_csv(f"submission_{icount}.csv",index=False)
sub2 = te_sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)